In [11]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r requirements_tf.txt

# Restart the runtime by ending the process.
exit()

In [1]:
import os
import matplotlib.pyplot as plt

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages, 1 = INFO, 2 = WARNING, 3 = ERROR
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import storage
import google.auth
from importlib import reload
from serving.constants import  BUCKET, IMG_SOURCE_PREFIX, HIST_DEST_PREFIX, NUM_BANDS, HIST_BINS_LIST, SCALE, CROP, MONTHS, IMAGE_BATCH, NUM_BINS
from serving.common import list_blobs_with_prefix
from serving.hist_training import create_hist_dataset, train_and_evaluate, create_data_sample, get_labels #, LstmModel
from serving.data import check_blob_prefix_exists, batch_check_blobs, get_varied_labels
import logging
import io
import itertools
from google.cloud import storage
import google.auth
import time

2024-10-23 00:34:18.814243: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-23 00:34:18.864143: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(features, label):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'feature': _bytes_feature(tf.io.serialize_tensor(features)),
        'label': _float_feature(label),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def save_dataset_to_gcp(dataset, bucket_name='vgnn', file_name='hist_dataset_medium.tfrecords'):
    # Initialize GCP client
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Create a local temporary file
    local_file_name = 'temp_' + file_name
    
    # Write the dataset to the local file
    with tf.io.TFRecordWriter(local_file_name) as writer:
        for features, label in dataset:
            example = serialize_example(features, label)
            writer.write(example)
    
    # Upload the local file to GCS
    blob = bucket.blob(f"dataset/{file_name}")
    blob.upload_from_filename(local_file_name)

    # Remove the local temporary file
    os.remove(local_file_name)

    print(f"Dataset saved to gs://{bucket_name}/dataset/{file_name}")
    
def parse_tfrecord_fn(example_proto):
    # Define the features dictionary that matches the structure used when saving
    feature_description = {
        'feature': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([1],tf.float32)
    }
    
    # Parse the input tf.Example proto using the feature description
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode the feature from the parsed example
    feature = tf.io.parse_tensor(parsed_features['feature'], out_type=tf.float32)
    label = parsed_features['label']
    
    return feature, label

def load_dataset_from_gcp(bucket_name='vgnn', file_name='hist_dataset_medium.tfrecords'):
    # Construct the full GCS path
    gcs_path = f"gs://{bucket_name}/dataset/{file_name}"
    
    # Create a TFRecordDataset
    dataset = tf.data.TFRecordDataset(gcs_path)
    
    # Parse the TFRecords
    parsed_dataset = dataset.map(parse_tfrecord_fn)     
    
    return parsed_dataset    

In [3]:
# SETUP

CREATE_DATASET = False

start_time = time.perf_counter()
credentials, _ = google.auth.default()
bucket_name = BUCKET
directory_prefix = IMG_SOURCE_PREFIX
output_prefix = HIST_DEST_PREFIX
labels_data = "labels_combined.npy"
labels_header = "labels_header.npy"
batch_size = 64

client = storage.Client()
bucket = client.get_bucket(bucket_name)



In [4]:
if CREATE_DATASET:
    # Generate all prefixes
    start_time = time.perf_counter()
    
    labels_df = get_varied_labels(0,10)
    labels = list(zip(labels_df["county_name"],
                      labels_df["county_ansi"],
                 labels_df["state_ansi"],
                 labels_df["year"]))


    prefixes_hist = [f'histograms/{NUM_BINS}_bcukets/{SCALE}/{county.capitalize()}_{fips}/{year}' 
                for county,_, fips, year in labels]

    # Batch check all prefixes
    results_hist = batch_check_blobs(bucket_name, prefixes_hist)

    # Generate get_input_img_params based on results
    get_input_img_params = [(county.capitalize(), fips, year)
                for county, county_fips, fips, year in labels
                if results_hist[f'histograms/{NUM_BINS}_bcukets/{SCALE}/{county.capitalize()}_{fips}/{year}']
    ]

    print(f"Number of items to process: {len(get_input_img_params)}")
    
    end_time_dataset_info = time.perf_counter()
    print(f"Collecting dataset information: {end_time_dataset_info-start_time}")
    
    dataset, input_shape = create_hist_dataset(get_input_img_params, "labels_combined.npy", "labels_header.npy")
    end_time_dataset = time.perf_counter()
    
    print(f"Building dataset: {(end_time_dataset - end_time_dataset_info)/60:.02} minutes")
    
    # Save dataset
    save_dataset_to_gcp(dataset, file_name = f"{NUM_BINS}_buckets_training_test")
    print("Dataset created")
else:
    dataset = load_dataset_from_gcp(file_name=f"{NUM_BINS}_buckets_training_test")
    for features, label in dataset.take(1):
        input_shape = tuple(features.shape)
        label_shape = label.shape
    print("Dataset loaded")

Dataset loaded


2024-10-23 00:34:24.012784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


In [5]:
for features, label in dataset.take(1):
    input_shape = tuple(features.shape)

print(f"""input_shape :{input_shape}
dataset: {type(dataset)}""")

input_shape :(3, 1664)
dataset: <class 'tensorflow.python.data.ops.map_op._MapDataset'>


2024-10-23 00:34:24.492292: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


In [6]:
label_data = np.load(labels_data, allow_pickle=True)
label_header = np.load(labels_header, allow_pickle=True)
label_df = pd.DataFrame(label_data, columns=label_header)
label_df[label_df["county_name"]=='Sutter'.upper()]

,commodity_desc,reference_period_desc,year,state_ansi,state_name,county_ansi,county_name,target,source_file
66,CORN,YEAR,2016,06,CALIFORNIA,101,SUTTER,264.1,USDA_Corn_County_2016.csv
3045,CORN,YEAR,2018,06,CALIFORNIA,101,SUTTER,210.0,USDA_Corn_County_2018.csv
4389,CORN,YEAR,2019,06,CALIFORNIA,101,SUTTER,147.1,USDA_Corn_County_2019.csv
7314,CORN,YEAR,2021,06,CALIFORNIA,101,SUTTER,169.9,USDA_Corn_County_2021.csv
8791,CORN,YEAR,2022,06,CALIFORNIA,101,SUTTER,133.8,USDA_Corn_County_2022.csv


In [7]:
# Function to check shapes
def check_dataset_shapes(dataset, num_samples_to_check=5):
    for i, (features, label) in enumerate(dataset.take(num_samples_to_check)):
        print(f"Sample {i+1}:")
        print(f"  Features shape: {features.shape}")
        print(f"  Label shape: {label.shape}")
        if i == 0:
            first_shape = features.shape
        else:
            if features.shape != first_shape:
                print("Warning: Inconsistent feature shape detected!")
                break

# Check the shapes of the first few samples
check_dataset_shapes(dataset)

Sample 1:
  Features shape: (3, 1664)
  Label shape: (1,)
Sample 2:
  Features shape: (3, 1664)
  Label shape: (1,)
Sample 3:
  Features shape: (3, 1664)
  Label shape: (1,)
Sample 4:
  Features shape: (3, 1664)
  Label shape: (1,)
Sample 5:
  Features shape: (3, 1664)
  Label shape: (1,)


2024-10-23 00:34:30.831107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


In [11]:
import keras
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow
import numpy as np
from keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import mlflow
import mlflow.keras
import matplotlib.pyplot as plt

def set_seeds(seed=42):
    # Set seeds for Python's random module
    random.seed(seed)

    # Set seed for NumPy
    np.random.seed(seed)

    # Set seed for TensorFlow
    tf.random.set_seed(seed)

  

# Define the LSTM model
class LstmModel(keras.Model):
    def __init__(self, input_shape, lstm_layers=3, no_units=3, output_units=1, dropout_rate=0.2, mean_response=0):
        super(LstmModel, self).__init__()
        self.lstm_layers = lstm_layers
        self.no_units = no_units
        self.output_units = output_units
        self.dropout_rate = dropout_rate
        self.mean_response = mean_response

        # Define LSTM and Dense layers
        self.lstm_layers_list = [
            LSTM(units=no_units, return_sequences=(i < lstm_layers - 1), 
                 kernel_initializer='zeros', recurrent_initializer='zeros', bias_initializer='zeros')
            for i in range(lstm_layers)
        ]
        self.dense = Dense(units=output_units, kernel_initializer='zeros', 
                           bias_initializer=tf.keras.initializers.Constant(mean_response))
    
    @tf.function
    def call(self, inputs, training=False):
        #LSTM layers
        x = inputs
        
        for lstm_layer in self.lstm_layers_list:
            x = lstm_layer(x)
            
        outputs = self.dense(x)
        
        return outputs

    def summary(self):
        super(LstmModel, self).summary()
    
    def compile(self, optimizer='adam', loss='mse', metrics=['mse'], learning_rate=0.001):
        self.learning_rate = learning_rate

        # Dictionary to map optimizer names to their classes
        optimizers = {
            'adam': Adam,
            'nadam': Nadam,
            'rms': RMSprop
        }

        # Get the optimizer class from the dictionary
        optimizer_class = optimizers.get(optimizer.lower(), Adam)

        # Instantiate the optimizer with the specified learning rate
        optimizer_instance = optimizer_class(learning_rate=learning_rate)

        # Compile the model with the chosen optimizer, loss, and metrics
        tf.config.run_functions_eagerly(True)
        super(LstmModel, self).compile(optimizer=optimizer_instance, loss=loss, metrics=metrics)
        
    def fit(self, dataset, epochs=10, batch_size=32):
        # Set up MLflow experiment
        mlflow.set_experiment('LSTM_Experiment')
        
        # Shuffle and batch the dataset
        dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)

        # Split the dataset
        val_size = 5
        val_dataset = dataset.take(val_size)
        train_dataset = dataset.skip(val_size)        

        # Start MLflow run
        with mlflow.start_run():
            # Early stopping callback
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            # Create a history callback
            history_callback = tf.keras.callbacks.History()
           
            # Train the model
            history = super(LstmModel, self).fit(train_dataset, epochs=epochs, batch_size=batch_size,
                           validation_data=val_dataset, callbacks=[early_stopping, history_callback])

            # Log model and parameters to MLflow
            mlflow.keras.log_model(self, 'model')
            mlflow.log_param('epochs', epochs)
            mlflow.log_param('batch_size', batch_size)
            mlflow.log_param('lstm_layers', self.lstm_layers)
            mlflow.log_param('learning_rate', self.learning_rate)
            
            # Plot training progress
            self.plot_training_progress(history)            

            # Evaluate the model
            loss = super(LstmModel, self).evaluate(val_dataset)
            mlflow.log_metric('val_loss', loss)

    def plot_training_progress(self, history):
        # Plot training and validation loss
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Training and Validation Loss')

        plt.tight_layout()
        plt.show()        

In [12]:
responses = []
for _, response in dataset:
    responses.append(response.numpy())
mean_response = np.mean(responses)
print(mean_response)

model = LstmModel(input_shape=input_shape, lstm_layers=7, no_units=5, mean_response=mean_response)
model.compile(optimizer='adam')
model.fit(dataset, epochs=100, batch_size=batch_size)

134.7283
Epoch 1/100


2024-10-23 00:39:05.173497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2024-10-23 00:39:05.173790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
plt.ion()
plt.figure(figsize=(10, 5))
plt.plot(history.history["loss"], label="Training Loss", color="blue")
plt.plot(history.history["val_loss"], label="Validation Loss", color="orange")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()